In [1]:
# Download libraries
!pip install azure-storage-blob # Microsoft Azure

In [2]:
# Import libraries
from io import StringIO
import pandas as pd
import numpy as np
import json
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient


In [3]:
# Extracting from datasource
# The URL from which to download the CSV
URL = "https://data.cityofnewyork.us/api/views/9ck6-2jew/rows.csv?fourfour=9ck6-2jew&cacheBust=1689185598&date=20240328&accessType=DOWNLOAD"

# Send a GET request to the URL
response = requests.get(URL)

# Check if the request was successful
if response.status_code == 200:
    # Open a file in binary write mode
    with open('rentalincome.csv', 'wb') as file:
        # Write the content of the response to the file
        file.write(response.content)
    print("CSV file has been downloaded and saved as 'rentalincome.csv'.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

CSV file has been downloaded and saved as 'rentalincome.csv'.


In [4]:
# Load the CSV file into a DataFrame
df_raw = pd.read_csv('rentalincome.csv')
print(df_raw.info())
print(df_raw.shape)
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31575 entries, 0 to 31574
Data columns (total 61 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Boro-Block-Lot                  31575 non-null  object 
 1   Condo Section                   31575 non-null  object 
 2   Address                         31575 non-null  object 
 3   Neighborhood                    31573 non-null  object 
 4   Building Classification         31575 non-null  object 
 5   Total Units                     31575 non-null  int64  
 6   Year Built                      31500 non-null  float64
 7   Gross SqFt                      31575 non-null  int64  
 8   Estimated Gross Income          31575 non-null  int64  
 9   Gross Income per SqFt           31575 non-null  float64
 10  Estimated Expense               31574 non-null  float64
 11  Expense per SqFt                31574 non-null  float64
 12  Net Operating Income            

,Boro-Block-Lot,Condo Section,Address,Neighborhood,Building Classification,Total Units,Year Built,Gross SqFt,Estimated Gross Income,Gross Income per SqFt,...,Gross SqFt 3,Estimated Gross Income 3,Gross Income per SqFt 3,Estimated Expense 3,Expense per SqFt 3,Net Operating Income 3,Full Market Value 3,Market Value per SqFt 3,Distance from Condo in miles 2,Report Year
0,1-00576-7501,0003-R1,60 WEST 13 STREET,GREENWICH VILLAGE-CENTRAL,R4 -ELEVATOR,70,1966.0,82017,4452703,54.29,...,27108.0,1471693.0,54.29,571708.0,21.09,899985.0,7318000.0,269.96,0.35,2019
1,1-01271-7501,0007-R2,1360 6 AVENUE,MIDTOWN WEST,R4 -ELEVATOR,183,1963.0,141738,7113830,50.19,...,39510.0,1987748.0,50.31,527854.0,13.36,1459894.0,9322000.0,235.94,0.63,2019
2,1-00894-7501,0009-R1,77 PARK AVENUE,MURRAY HILL,R4 -ELEVATOR,109,1924.0,158571,7329152,46.22,...,20784.0,960636.0,46.22,374112.0,18.00,586524.0,3423000.0,164.69,0.22,2019
3,1-00631-7501,0018-R1,712 GREENWICH STREET,GREENWICH VILLAGE-WEST,R9 -CONDOPS,20,1910.0,53943,2132906,39.54,...,37064.0,1461804.0,39.44,504070.0,13.60,957734.0,14265000.0,384.87,0.19,2019
4,1-00868-7501,0019-R1,35 EAST 38 STREET,MURRAY HILL,R4 -ELEVATOR,113,1961.0,88230,4288860,48.61,...,41603.0,2022322.0,48.61,497572.0,11.96,1524750.0,7628000.0,183.35,0.22,2019


In [7]:
CONNECTION_STRING_AZURE_STORAGE = "DefaultEndpointsProtocol=https;AccountName=cis4400spring2024;AccountKey=kDPtmsrNnzE9S72dz+hvgYnvJCkAWw4y5JI5lbjVbYNQFdsEug60n4BUy0Pr5XrQRVKAe9f54cjK+AStCJo60g==;EndpointSuffix=core.windows.net"
CONTAINER_AZURE = 'rentalincome'
blob_name = "rentalincome.csv"

# Convert DataFrame to CSV
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded rentalincome.csv to Azure Blob Storage in container rentalincome.
